In [2]:
import pandas as pd
import numpy as np
import glob
import os

In [3]:
folder = r"C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Preparation\\dataset\\scaped_data"
files = glob.glob(os.path.join(folder, "*.csv"))


In [ ]:
all_titles = set()
skipped_files = []

for f in files:
    try:
 
        if os.path.getsize(f) == 0:
            print(f"Skipping empty file: {os.path.basename(f)}")
            skipped_files.append(f)
            continue

        df = pd.read_csv(f)

   
        if "Destination Title" not in df.columns:
            print(f"Missing column in: {os.path.basename(f)}")
            skipped_files.append(f)
            continue

     
        all_titles.update(df["Destination Title"].dropna().unique())

    except Exception as e:
        print(f"Error reading {os.path.basename(f)} → {e}")
        skipped_files.append(f)


unique_scraped_titles = sorted(all_titles)

pd.DataFrame({"scraped_title": unique_scraped_titles}) \
    .to_csv("unique_scraped_titles.csv", index=False)

print("\nDONE")
print(f"Total CSV files: {len(files)}")
print(f"Valid destination titles: {len(unique_scraped_titles)}")
print(f"Skipped files: {len(skipped_files)}")
# all_titles = set()

# for f in files:
#     df = pd.read_csv(f)
#     nul = df["Destination Title"].isnull().sum()
#     print(f"{f}: {nul} null values")
#     all_titles.update(df["Destination Title"].dropna().unique())
#     nul2 = df["Destination Title"].isnull().sum()
#     print(f"{f}: {nul2} null values")

# unique_scraped_titles = sorted(all_titles)

# pd.DataFrame({"scraped_title": unique_scraped_titles}) \
#   .to_csv("unique_scraped_titles.csv", index=False)

Error reading Archaeological MuseumUnclaimedIf you own this business, claim it for free now to update business info, respond to reviews, and more.Claim this listing.csv → [WinError 3] The system cannot find the path specified: 'C:\\\\NIBM\\\\Travel recommendation system\\\\Travel-Recommendation-system-in-Sri-Lanka-\\\\Data Preparation\\\\dataset\\\\scaped_data\\Archaeological MuseumUnclaimedIf you own this business, claim it for free now to update business info, respond to reviews, and more.Claim this listing.csv'

DONE
Total CSV files: 219
Valid destination titles: 218
Skipped files: 1


In [5]:
dest = pd.read_csv("C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Scraper\\dataset\\srilanka_destinations.csv", encoding="latin1")


In [6]:
dest.head()

,Column1,Column2,Column3,Column4,Column5
0,Destination,Category,Subcategory,Review Counts,Links
1,Unawatuna Beach,Beaches & Coastal Destinations,South Coast,188,https://www.tripadvisor.com/Attraction_Review-...
2,Mirissa Beach,Beaches & Coastal Destinations,South Coast,3157,https://www.tripadvisor.com/Attraction_Review-...
3,Weligama Beach,Beaches & Coastal Destinations,South Coast,231,https://www.tripadvisor.com/Attraction_Review-...
4,Hikkaduwa Beach,Beaches & Coastal Destinations,South Coast,2425,https://www.tripadvisor.com/Attraction_Review-...


In [7]:
dest.columns = dest.iloc[0]
dest = dest[1:]

In [8]:
dest_titles = dest["Destination"].dropna().unique()

pd.Series(dest_titles, name="Destination") \
  .to_csv("dest_titles.csv", index=False, encoding="utf-8")

In [9]:
df1 = pd.read_csv("unique_scraped_titles.csv", encoding="utf-8")
df2 = pd.read_csv("dest_titles.csv", encoding="utf-8")

In [10]:
df1

,scraped_title
0,Abayagiriya Stupa
1,Aberdeen Falls
2,Alagalla Mountain Range
3,Aluvihara Rock Cave Temple
4,Ambuluwawa Tower
...,...
213,Whale Watching Mirissa
214,Wilpattu National Park
215,World's End
216,Yala National Park


In [11]:
df2

,Destination
0,Unawatuna Beach
1,Mirissa Beach
2,Weligama Beach
3,Hikkaduwa Beach
4,Dalawella Beach
...,...
372,Negombo Fish Market Complex
373,Jaffna Market
374,Cargills Square â Jaffna
375,Manalkadu Beach


In [12]:
! pip install fuzzywuzzy[speedup]

In [13]:
import pandas as pd
import glob
import os

In [ ]:

source_folder = r"C:\NIBM\Travel recommendation system\Travel-Recommendation-system-in-Sri-Lanka-\dataset\scaped_data"
destination_folder = r"C:\NIBM\Travel recommendation system\Travel-Recommendation-system-in-Sri-Lanka-\dataset\updated_csvs"
os.makedirs(destination_folder, exist_ok=True)

df2 = pd.read_csv("dest_titles.csv", encoding="utf-8")
clean_destinations = df2["Destination"].dropna().tolist()


files = glob.glob(os.path.join(source_folder, "*.csv"))

def map_destination_by_prefix(title, prefix_len=30):
    if pd.isna(title):
        return title
    prefix = title[:prefix_len]  
    for clean_title in clean_destinations:
        if clean_title.startswith(prefix):
            return clean_title  
    return title  


for file_path in files:
    try:
        df = pd.read_csv(file_path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="latin1")
    except pd.errors.EmptyDataError:
        print(f"Skipping empty file: {file_path}")
        continue

    if "Destination Title" not in df.columns:
        print(f"Skipping file with no 'Destination Title': {file_path}")
        continue

    df["Destination Title"] = df["Destination Title"].apply(map_destination_by_prefix)

    # Save updated CSV
    filename = os.path.basename(file_path)
    new_file_path = os.path.join(destination_folder, filename)
    df.to_csv(new_file_path, index=False, encoding="utf-8")
    print(f"Processed and saved: {filename}")

print("✅ All files processed!")
